In [1]:
import copy
import numpy as np
import os
import pandas as pd
pd.options.display.max_rows = 8

## 使い方
- pathを変更する

In [2]:
paths = ['tfidf_text/0thLayer','tfidf_text/1stLayer','tfidf_text/2ndLayer']

In [3]:
all_nodes = set()
link_dict = dict()
link_df = pd.DataFrame(columns=['from','to','value'])
for path in paths:
    files = os.listdir(path)
    for file in files:
        query,_ = os.path.splitext(file)
        if not query.startswith('.'):
            all_nodes.add(query)
            link_dict[query] = []
            records = []
            file = open('{0}/{1}.txt'.format(path,query),'r')
            lines = file.readlines()
            for i in range(20):
                to_query = lines[i].split(',')[0][2:-1]
                value = float(lines[i].split(',')[1][1:-2])
                if not query == to_query:  #セルフノード削除
                    all_nodes.add(to_query)
                    link_dict[query].append(to_query)
                    records.append([query, to_query, value])
            df_ = pd.DataFrame(records,columns=link_df.columns)
            link_df = link_df.append(df_)

# link_df.drop(link_df[link_df["to"]==link_df["from"]].index) 
link_df = link_df.reset_index(drop=True)

In [4]:
#link_df.to_csv('links.csv',encoding='utf-8',index=False)

In [5]:
#link_df = pd.read_csv('links.csv')

初期化

In [6]:
len(link_dict)

570

In [7]:
len(all_nodes)

2798

In [8]:
len(link_df)

11392

In [9]:
tmp = pd.DataFrame()
tmp['from'] = link_df['to']
tmp['to'] = link_df['from']
tmp['value'] = link_df['value']
link_df = link_df.append(tmp)
link_df.reset_index(drop=True)

new_link_dict = copy.deepcopy(link_dict)
for key,value in link_dict.items():
    for val in value:
        if val in link_dict.keys():
            new_link_dict[val].append(key)
        else:
            new_link_dict[val] = [key]
link_dict = new_link_dict

In [10]:
univs = {
    "東京大学":"from_ut",
    "京都大学":"from_ky",
    "早稲田大学":"from_wa",
    "慶應義塾大学":"from_ko",
    "明治大学":"from_me"
}

df版

In [11]:
# node_df = pd.DataFrame(
#     index=range(len(all_nodes)), 
#     columns=[
#         'query',
#         'from_ut',
#         'from_ky',
#         'from_wa',
#         'from_ko',
#         'from_me',
# #         'label',
# #         'shortest_step'
#     ])
# node_df['query'] = list(all_nodes)
# node_df = node_df.fillna(float(99))

In [12]:
# for key, value in univs.items():
#     node_df.loc[node_df['query'] == key, value] = 0

In [13]:
# from_querys = univs.keys()

In [14]:
# next_step = link_df[link_df['from'].isin(from_querys)]

In [15]:
# for from_query in from_querys:
#     value = node_df.loc[node_df['query'] == from_query,univs.values()]
#     for to_query in link_dict[from_query]:
#         for column in univs.values():
#             node_df.loc[node_df['query'] == to_query, column] = value

dict版

In [16]:
def check_layer(from_querys,node_dict,link_dict,univs):
    for from_query in from_querys:
        distance_dict = copy.deepcopy(node_dict[from_query])
        for key in distance_dict.keys():
            distance_dict[key] += 1
        try:
            for to_query in link_dict[from_query]:
                for key in univs.values():
                    node_dict[to_query][key] = min(distance_dict[key],node_dict[to_query][key])
        except KeyError:
            pass
    
    return node_dict

In [17]:
def get_next_from_querys(link_df,from_querys):
    next_step = link_df[link_df['from'].isin(from_querys)]
    
    return list(next_step['to'])

In [18]:
node_dict = {}
for query in all_nodes:
    node_dict[query] = {
    "from_ut":99,
    "from_ky":99,
    "from_wa":99,
    "from_ko":99,
    "from_me":99
}

#初期化
node_dict["東京大学"]["from_ut"] = 0
node_dict["京都大学"]["from_ky"] = 0
node_dict["早稲田大学"]["from_wa"] = 0
node_dict["慶應義塾大学"]["from_ko"] = 0
node_dict["明治大学"]["from_me"] = 0

In [19]:
#1層目
from_querys = list(univs.keys())

In [20]:
for x in range(30):
    node_dict = check_layer(from_querys,node_dict,link_dict,univs)
    from_querys = get_next_from_querys(link_df,from_querys)

In [21]:
list_of_list = []
for key, value in node_dict.items():
    list_of_list.append([key,value['from_ut'],value['from_ky'],value['from_wa'],value['from_ko'],value['from_me']])
node_df = pd.DataFrame(list_of_list,columns=['query','from_ut','from_ky','from_wa','from_ko','from_me'])

In [22]:
node_df['shortest_step'] = np.min(node_df[list(univs.values())],axis=1)

In [23]:
for column in ['ut','ky','wa','ko','me']:
    node_df['label_{}'.format(column)] = node_df['from_{}'.format(column)] == node_df['shortest_step']

In [27]:
univ_df = pd.DataFrame()
univ_df = univ_df.append(node_df[node_df['query'] == '東京大学']).append(node_df[node_df['query'] == '京都大学'].append(node_df[node_df['query'] == '明治大学'])).append(node_df[node_df['query'] == '早稲田大学']).append(node_df[node_df['query'] == '慶應義塾大学'])

In [28]:
univ_df

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me
529,東京大学,0,2,2,2,2,0,True,False,False,False,False
2671,京都大学,2,0,2,2,2,0,False,True,False,False,False
1990,明治大学,2,2,2,2,0,0,False,False,False,False,True
935,早稲田大学,2,2,0,2,2,0,False,False,True,False,False
127,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False


In [31]:
node_df = node_df.drop(univ_df.index)

In [33]:
node_df = univ_df.append(node_df).reset_index(drop=True)

In [34]:
node_df['index'] = node_df.index

In [35]:
node_df

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,会食,4,4,4,4,3,3,False,False,False,False,True,2794
2795,スマートダイヤル,3,3,3,3,3,3,True,True,True,True,True,2795
2796,携帯,3,3,3,3,3,3,True,True,True,True,True,2796
2797,葬儀,3,3,2,2,2,2,False,False,True,True,True,2797


In [37]:
def get_node_index(x):
    number = node_df[node_df['query']==x]['index']
    return int(number)

In [38]:
link_df['from_id'] = link_df['from'].map(get_node_index)
link_df['to_id'] = link_df['to'].map(get_node_index)

In [39]:
link_df

,from,to,value,from_id,to_id
0,慶應義塾大学,研究,0.025952,4,1405
1,慶應義塾大学,学部,0.022282,4,439
2,慶應義塾大学,月,0.020959,4,2603
3,慶應義塾大学,学生,0.020782,4,1603
...,...,...,...,...,...
11388,ミニロト,宝くじ,0.034209,525,1597
11389,抽選,宝くじ,0.033720,219,1597
11390,ネット,宝くじ,0.031710,2793,1597
11391,ブロック,宝くじ,0.030341,1991,1597


In [40]:
# node_df.to_csv('nodes.csv')
# link_df.to_csv('links.csv')